In [1]:
import os, sys
import numpy as np
import re
import pandas as pd

import matplotlib
%matplotlib inline
#matplotlib.use('Agg')
import matplotlib.pyplot as plt

plt.style.use('ggplot')

caffe_root = '/home/ashu/Desktop/Thesis Work/Classifier/caffe/'
print os.getcwd()
sys.path.insert(0, caffe_root + 'python')
import caffe

import argparse

from caffe.proto import caffe_pb2
import google.protobuf as pb2
import google.protobuf.text_format

import lmdb
from collections import defaultdict
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

/home/ashu/Desktop/Thesis Work/Classifier/caffe/models/script


- Load the labels (val.txt)

In [26]:
#Set the path to labels file
cls_lbl_file = caffe_root + 'data/gt_data/labels/suitcase_val.txt'
# read the labels file
with open(cls_lbl_file, 'r') as f:
    cls_data=f.readlines()
    
cls_data = [x.strip() for x in cls_data]

# append image name and its label in a dictionary
cls_lbl = {}
for val in cls_data:
    img, lbl = val.split(' ')
    cls_lbl[img] = lbl
    
print len(cls_data), len(cls_lbl)

2216 2216


- create prototxt and caffe model filename
- create the transformer as well

- create 2 list to carry tp and fp for each class
- 2D list indexed acc to class label
- loop through each file in val txt, load class label and feed fwd image
- take predictions and acc to class label update app tp and fp
- end all list ll be of unequal size
- cal prec and recall at each point and finally compute avg precision

In [27]:
caffe.set_mode_gpu()

img_dir = caffe_root + 'data/gen_data/images/'

# Modify the paths given below
deploy_prototxt_file_path = caffe_root + 'models/gt_classifier/gen_suitcase/test.prototxt' # Network definition file

max_iter = 40000
best_iter = 22000

caffe_model_file_path = caffe_root + 'models/gt_classifier/gen_suitcase/trained_model' + \
                            '/gen_suitcase_' +  str(max_iter) + \
                            '_iter_' + str(best_iter) + '.caffemodel'
    
net = caffe.Net(deploy_prototxt_file_path, caffe_model_file_path, caffe.TEST)
    
# set the transformeer
# load the mean ImageNet image (as distributed with Caffe) for subtraction
mu = np.load(caffe_root + 'python/caffe/imagenet/ilsvrc_2012_mean.npy')
mu = mu.mean(1).mean(1)  # average over pixels to obtain the mean (BGR) pixel values
print 'mean-subtracted values:', zip('BGR', mu)

# create transformer for the input called 'data'
transformer = caffe.io.Transformer({'data': net.blobs['data'].data.shape})

transformer.set_transpose('data', (2,0,1))  # move image channels to outermost dimension
transformer.set_mean('data', mu)            # subtract the dataset-mean value in each channel
transformer.set_raw_scale('data', 255)      # rescale from [0, 1] to [0, 255]
transformer.set_channel_swap('data', (2,1,0))  # swap channels from RGB to BGR

net.blobs['data'].reshape(1,        # batch size
                          3,         # 3-channel (BGR) images
                          224, 224)  # image size is 227x227
    
labels = []
plabels = []

for k,v in cls_lbl.iteritems():
        
    # load image, transform and feed forward
    image = caffe.io.load_image(img_dir + k)
    transformed_image = transformer.preprocess('data', image)
    net.blobs['data'].data[...] = transformed_image
    output = net.forward()
    plabel = int(output['prob'][0].argmax(axis=0))
    label = int(v)
    labels.append(label)
    plabels.append(plabel)

mean-subtracted values: [('B', 104.0069879317889), ('G', 116.66876761696767), ('R', 122.6789143406786)]


In [28]:
print 'Accuracy Closed : ', accuracy_score(labels, plabels)

Accuracy Closed :  0.291967509025


In [29]:
cmat = confusion_matrix(labels, plabels)
print cmat

[[ 134   17]
 [1552  513]]


In [30]:
print cmat.diagonal()/cmat.sum(axis=1, dtype=np.float32)

[ 0.88741722  0.24842615]


In [31]:
print classification_report(labels, plabels, target_names=['Open', 'Closed'])

             precision    recall  f1-score   support

       Open       0.08      0.89      0.15       151
     Closed       0.97      0.25      0.40      2065

avg / total       0.91      0.29      0.38      2216

